# Advertisement Marketing: Regression

In [49]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## 1. Dataset 불러오기

In [14]:
# 전처리단계

# Assuming your data is stored in an Excel file named "VMI_data.xls"
data = pd.read_excel("./dataset/VMI_data.xls", index_col="Part Number")

# Reshape the data using the pivot_table function
reshaped_data = data.pivot_table(index=data.index, columns="CRET_TIME", values="D일 투입예정 수량(D일계획)", fill_value=0)

# Set the column labels to the CRET_TIME values
reshaped_data.columns = reshaped_data.columns.rename(None)

# Reset the index to get "Part Number" as a column
reshaped_data.reset_index(inplace=True)

# Save the reshaped data to a new CSV file
reshaped_data.to_csv("reshaped_data.csv", index=False)


In [53]:
# 전처리단계
data = pd.read_excel("./dataset/VMI_data.xls",index_col='Part Number')
data['CRET_TIME'] = pd.to_datetime(data['CRET_TIME'], format='%Y%m%d%H%M')
data.head()

,D일06~08(08)H 투입계획(발주) 수량,D일08~10(10)H 투입계획(발주) 수량,D일10~12(13)H 투입계획(발주) 수량,D일13~15(15)H 투입계획(발주) 수량,D일15~17(18)H 투입계획(발주) 수량,D일18~20(21)H 투입계획(발주) 수량,D일21~23(23)H 투입계획(발주) 수량,D일23~01(02)H 투입계획(발주) 수량,D일 02~04H 투입계획 수량,D일 04~06H 투입계획 수량,...,D+23일 투입예정 수량,D+24일 투입예정 수량,D+25일 투입예정 수량,D+26일 투입예정 수량,D+27일 투입예정 수량,D+28일 투입예정 수량,D+29일 투입예정 수량,D+30일 투입예정 수량,D+31~D+45일 투입예정 수량,CRET_TIME
Part Number,,,,,,,,,,,,,,,,,,,,,
Part 0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2021-09-13 18:30:00
Part 1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2021-09-13 18:30:00
Part 2,139,60,80,76,68,81,68,44,0,0,...,0,0,0,0,0,0,0,0,0,2021-09-13 18:30:00
Part 3,15,11,17,13,12,16,12,8,0,0,...,0,0,0,0,0,0,0,0,0,2021-09-13 18:30:00
Part 4,40,27,29,17,18,29,18,9,0,0,...,0,0,0,0,0,0,0,0,0,2021-09-13 18:30:00


In [55]:
# Reshape the data using the pivot_table function
reshaped_data = data.pivot_table(index=data.index, columns="CRET_TIME", values="D일 투입예정 수량(D일계획)", fill_value=0)

# Set the column labels to the CRET_TIME values
reshaped_data.columns = reshaped_data.columns.rename(None)

# Reset the index to get "Part Number" as a column
reshaped_data.reset_index(inplace=True)

# Save the reshaped data to a new CSV file
reshaped_data.to_csv("reshaped_data.csv", index=False)

In [59]:
df = pd.read_csv("reshaped_data.csv",index_col='Part Number')
df

,2021-09-13 18:30:00,2021-09-14 06:05:00,2021-09-14 06:25:00,2021-09-14 06:34:00,2021-09-14 17:30:00,2021-09-15 06:20:00,2021-09-15 06:37:00,2021-09-15 16:33:00,2021-09-16 06:32:00,2021-09-16 06:38:00,...,2021-10-28 15:30:00,2021-10-28 16:26:00,2021-10-29 06:30:00,2021-10-29 06:32:00,2021-10-29 07:06:00,2021-10-29 12:34:00,2021-10-29 15:03:00,2021-10-29 15:38:00,2021-10-30 07:04:00,2021-11-01 07:03:00
Part Number,,,,,,,,,,,,,,,,,,,,,
Part 0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0.0
Part 1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0.0
Part 10,16,16,16.0,16.0,16.0,16.0,16.0,16.0,23.0,23.0,...,178.0,178.0,178.0,156,156,156,156,156,156,104.0
Part 100,38,38,35.0,35.0,35.0,40.0,40.0,40.0,38.0,38.0,...,33.0,33.0,33.0,23,23,23,23,23,23,0.0
Part 101,0,0,0.0,0.0,0.0,77.0,77.0,77.0,78.0,78.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Part 95,113,113,209.0,209.0,209.0,209.0,209.0,209.0,288.0,288.0,...,276.0,276.0,276.0,264,264,264,264,264,264,209.0
Part 96,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,6.0,6.0,0,0,0,0,0,0,0.0
Part 97,127,127,32.0,32.0,32.0,28.0,28.0,28.0,28.0,28.0,...,4.0,4.0,4.0,3,3,3,3,3,3,1.0
